# CENSUS DATA


# EDA

## Data Exploration

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


### Import data

##### Zip Code Data
- using for city information

In [3]:
uszip = pd.read_csv('/content/uszips.csv')

In [4]:
print(uszip.shape)
uszip.head()

(33121, 18)


,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18005,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17113.0,102.7,72001,Adjuntas,"{""72001"": ""99.43"", ""72141"": ""0.57""}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36074,-67.17519,Aguada,PR,Puerto Rico,True,NaN,37751.0,476.0,72003,Aguada,"{""72003"": ""100""}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45440,-67.12201,Aguadilla,PR,Puerto Rico,True,NaN,47081.0,574.9,72005,Aguadilla,"{""72005"": ""100""}",Aguadilla,72005,False,False,America/Puerto_Rico
3,606,18.16721,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6392.0,58.3,72093,Maricao,"{""72093"": ""94.88"", ""72153"": ""3.78"", ""72121"": ""...",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29032,-67.12244,Anasco,PR,Puerto Rico,True,NaN,26686.0,286.9,72011,Añasco,"{""72011"": ""99.45"", ""72003"": ""0.55""}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


In [5]:
uszip['zip'] = uszip['zip'].astype('str').str.zfill(5)

In [6]:
uszip['zip'].nunique()

33121

##### Census 
- Contains population, demographic, poverty, and employment data by tract

In [7]:
df = pd.read_csv('/content/acs2017_census_tract_data.csv')
print(df.shape)
df.head()

(74001, 37)


,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,1.2,0.0,1407,67826.0,14560.0,33018.0,6294.0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806.0,9496.0,21236.0,2562.0,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,0.2,0.0,3257,55895.0,4369.0,28068.0,3190.0,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,3.1,0.0,7229,68143.0,14424.0,36905.0,10706.0,12.2,17.9,48.8,13.8,20.5,3.5,13.4,86.9,11.2,0.0,0.8,0.3,0.7,21.0,4787,71.4,24.1,4.5,0.0,2.3


In [8]:
df['TractId'].nunique()

74001

##### Tract to Zip
- maps census tracts to zip codes

In [9]:
tract_zip = pd.read_excel('/content/TRACT_ZIP_122017.xlsx')

In [10]:
print(tract_zip.shape)
tract_zip.head()

(168662, 6)


,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,1001020100,36067,1.000000,1.000000,1.00,1.000000
1,1001020200,36008,0.028243,0.012987,0.00,0.023846
2,1001020200,36067,0.971757,0.987013,1.00,0.976154
3,1001020300,36067,1.000000,1.000000,1.00,1.000000
4,1001020400,36067,0.025543,0.585987,0.25,0.071038


In [11]:
tract_zip = tract_zip[['tract', 'zip', 'res_ratio']]

In [12]:
tract_zip = tract_zip.rename(columns = {'tract':'TractId'})

In [13]:
tract_zip['zip'].nunique()

39301

In [14]:
tract_zip['TractId'].nunique()

73584

## Merge Census with Tract_Zip Data

In [15]:
merge = df.merge(tract_zip, on=['TractId'])

In [16]:
print(merge.shape)
merge.head()

(168385, 39)


,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,1.2,0.0,1407,67826.0,14560.0,33018.0,6294.0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6,36067,1.000000
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4,36008,0.028243
2,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4,36067,0.971757
3,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806.0,9496.0,21236.0,2562.0,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7,36067,1.000000
4,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,0.2,0.0,3257,55895.0,4369.0,28068.0,3190.0,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1,36067,0.025543


### Seeing what does not transfer over

In [17]:
df[(~df.TractId.isin(merge.TractId))]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
43,1003990000,Alabama,Baldwin County,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
868,1097990000,Alabama,Mobile County,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1266,2105000200,Alaska,Hoonah-Angoon Census Area,62,35,27,0.0,11.3,0.0,82.3,0.0,0.0,60,42500.0,9004.0,23827.0,3232.0,6.5,50.0,39.3,32.1,7.1,10.7,10.7,51.9,0.0,0.0,25.9,0.0,22.2,13.8,28,32.1,35.7,32.1,0.0,12.5
1324,2185000100,Alaska,North Slope Borough,4444,2366,2078,5.3,13.3,0.3,59.4,10.5,3.3,2769,82964.0,13376.0,31860.0,5094.0,11.2,13.0,35.6,15.0,26.4,7.4,15.6,48.1,26.8,0.5,11.9,9.0,3.8,6.6,1800,44.3,54.4,1.3,0.0,14.2
1353,4001944202,Arizona,Apache County,4094,1933,2161,3.4,0.5,0.4,93.9,0.2,0.0,2561,26296.0,6947.0,10079.0,1702.0,53.9,70.0,43.8,17.6,12.8,13.4,12.3,73.1,13.7,2.1,7.5,0.0,3.5,22.8,851,39.4,56.9,3.8,0.0,28.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73982,72151950700,Puerto Rico,Yabucoa Municipio,4744,2377,2367,100.0,0.0,0.0,0.0,0.0,0.0,4061,16087.0,3373.0,8157.0,1248.0,53.6,42.2,41.0,12.1,22.1,12.9,11.9,85.5,14.5,0.0,0.0,0.0,0.0,33.9,1034,61.4,36.2,2.4,0.0,17.9
73983,72151950800,Puerto Rico,Yabucoa Municipio,3531,1915,1616,100.0,0.0,0.0,0.0,0.0,0.0,2924,15767.0,2603.0,8133.0,1350.0,42.4,52.7,12.7,31.9,21.2,9.2,25.0,81.1,13.7,0.0,2.7,2.5,0.0,25.0,780,71.0,18.2,10.8,0.0,30.4
73984,72151950900,Puerto Rico,Yabucoa Municipio,5822,2892,2930,99.7,0.3,0.0,0.0,0.0,0.0,4380,13841.0,3098.0,7088.0,1162.0,58.7,66.5,29.0,24.1,25.5,9.2,12.2,95.7,2.0,0.0,2.3,0.0,0.0,32.3,1143,66.0,29.4,4.6,0.0,28.7
73989,72151990000,Puerto Rico,Yabucoa Municipio,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [18]:
tract_zip[(~tract_zip.TractId.isin(merge.TractId))]

,TractId,zip,res_ratio
543,1037030200,36080,1.0
2828,1115050101,35146,1.0
3017,1123231500,36866,1.0
3673,2270000100,99563,0.0
3674,2270000100,99620,0.0
...,...,...,...
168657,78030960800,802,1.0
168658,78030960900,802,1.0
168659,78030961000,802,1.0
168660,78030961100,802,1.0


## Data Cleaning

### isnull().sum()

In [19]:
merge.isnull().sum()

TractId                0
State                  0
County                 0
TotalPop               0
Men                    0
Women                  0
Hispanic             506
White                506
Black                506
Native               506
Asian                506
Pacific              506
VotingAgeCitizen       0
Income              1291
IncomeErr           1291
IncomePerCap         596
IncomePerCapErr      596
Poverty              710
ChildPoverty        1286
Professional         652
Service              652
Office               652
Construction         652
Production           652
Drive                633
Carpool              633
Transit              633
Walk                 633
OtherTransp          633
WorkAtHome           633
MeanCommute          949
Employed               0
PrivateWork          652
PublicWork           652
SelfEmployed         652
FamilyWork           652
Unemployment         650
zip                    0
res_ratio              0
dtype: int64

### Fill na with 0

In [20]:
merge.fillna(0, inplace=True)

In [21]:
merge.isnull().sum()

TractId             0
State               0
County              0
TotalPop            0
Men                 0
Women               0
Hispanic            0
White               0
Black               0
Native              0
Asian               0
Pacific             0
VotingAgeCitizen    0
Income              0
IncomeErr           0
IncomePerCap        0
IncomePerCapErr     0
Poverty             0
ChildPoverty        0
Professional        0
Service             0
Office              0
Construction        0
Production          0
Drive               0
Carpool             0
Transit             0
Walk                0
OtherTransp         0
WorkAtHome          0
MeanCommute         0
Employed            0
PrivateWork         0
PublicWork          0
SelfEmployed        0
FamilyWork          0
Unemployment        0
zip                 0
res_ratio           0
dtype: int64

In [22]:
merge.State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

### Remove nonstates

In [23]:
# Remove areas we don't have in other df
merge = merge[merge['State'] != 'District of Columbia']
merge = merge[merge['State'] != 'Puerto Rico']

In [24]:
merge['State'].nunique()

50

## Map Tracts to Zip Code
* first get multiply % columns by total population
* multiply residential ratio across rows

### Make Numbers Compatible

#### Dividing certain columns by 100


##### Making sure I capture the correct columns

In [25]:
merge.iloc[:,6:11+1]

,Hispanic,White,Black,Native,Asian,Pacific
0,2.4,86.3,5.2,0.0,1.2,0.0
1,1.1,41.6,54.5,0.0,1.0,0.0
2,1.1,41.6,54.5,0.0,1.0,0.0
3,8.0,61.4,26.5,0.6,0.7,0.4
4,9.6,80.3,7.1,0.5,0.2,0.0
...,...,...,...,...,...,...
166947,3.0,89.4,1.0,0.1,3.0,0.0
166948,3.0,89.4,1.0,0.1,3.0,0.0
166949,3.0,89.4,1.0,0.1,3.0,0.0
166950,3.0,89.4,1.0,0.1,3.0,0.0


In [26]:
merge.iloc[:,17:36+1]

,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,22.4,35.8,30.5,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
3,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
4,2.3,1.7,29.0,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166947,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4
166948,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4
166949,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4
166950,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4


In [27]:
merge.iloc[:,6:11+1] /= 100

In [28]:
merge.iloc[:,17:36+1] /= 100

In [29]:
merge.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
0,1001020100,Alabama,Autauga County,1845,899,946,0.024,0.863,0.052,0.000,0.012,0.000,1407,67826.0,14560.0,33018.0,6294.0,0.107,0.208,0.385,0.156,0.228,0.108,0.124,0.942,0.033,0.0,0.005,0.000,0.021,0.245,8.81,0.742,0.212,0.045,0.000,0.046,36067,1.000000
1,1001020200,Alabama,Autauga County,2172,1167,1005,0.011,0.416,0.545,0.000,0.010,0.000,1652,41287.0,3819.0,18996.0,2453.0,0.224,0.358,0.305,0.249,0.229,0.063,0.154,0.905,0.091,0.0,0.000,0.005,0.000,0.222,8.52,0.759,0.150,0.090,0.000,0.034,36008,0.028243
2,1001020200,Alabama,Autauga County,2172,1167,1005,0.011,0.416,0.545,0.000,0.010,0.000,1652,41287.0,3819.0,18996.0,2453.0,0.224,0.358,0.305,0.249,0.229,0.063,0.154,0.905,0.091,0.0,0.000,0.005,0.000,0.222,8.52,0.759,0.150,0.090,0.000,0.034,36067,0.971757
3,1001020300,Alabama,Autauga County,3385,1533,1852,0.080,0.614,0.265,0.006,0.007,0.004,2480,46806.0,9496.0,21236.0,2562.0,0.147,0.211,0.279,0.194,0.333,0.099,0.096,0.883,0.084,0.0,0.010,0.008,0.015,0.231,14.82,0.733,0.211,0.048,0.007,0.047,36067,1.000000
4,1001020400,Alabama,Autauga County,4267,2001,2266,0.096,0.803,0.071,0.005,0.002,0.000,3257,55895.0,4369.0,28068.0,3190.0,0.023,0.017,0.290,0.166,0.258,0.091,0.195,0.823,0.112,0.0,0.015,0.029,0.021,0.259,18.49,0.758,0.197,0.045,0.000,0.061,36067,0.025543


##### Multiply Columns by Total Population

In [30]:
merge.columns

Index(['TractId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'zip', 'res_ratio'],
      dtype='object')

In [31]:
col = ['Hispanic','White', 'Black', 'Native', 'Asian', 'Pacific','Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment']

In [32]:
merge[col] = merge[col].multiply(merge['TotalPop'], axis = 'index')

In [33]:
merge[col] = merge[col].astype(int)

##### Check values

In [34]:
df.loc[
       (df.State == 'Wyoming')
       &(df.TotalPop == 3329)
       &(df.TractId == 56045951100)]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
73054,56045951100,Wyoming,Weston County,3329,1946,1383,3.0,89.4,1.0,0.1,3.0,0.0,2700,62435.0,9525.0,32665.0,4047.0,12.5,18.5,32.2,15.9,13.4,20.5,17.9,72.2,8.0,9.5,2.1,3.0,5.2,31.3,1583,64.6,22.9,10.9,1.6,0.4


In [35]:
merge.loc[166948:166948]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
166948,56045951100,Wyoming,Weston County,3329,1946,1383,99,2976,33,3,99,0,2700,62435.0,9525.0,32665.0,4047.0,416,615,1071,529,446,682,595,2403,266,316,69,99,173,1041,52698,2150,762,362,53,13,82729,0.002424


In [36]:
int(3/100 * 3329)

99

In [37]:
int(89.4/100 * 3329)

2976

In [38]:
assert merge['Hispanic'][166948] == 99

In [39]:
assert merge['White'][166948] == 2976

In [40]:
df.loc[
       (df.State == 'Alabama')
       &(df.TotalPop == 7131)
       &(df.TractId == 1003011601)]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
41,1003011601,Alabama,Baldwin County,7131,3699,3432,9.7,87.6,1.4,0.7,0.1,0.0,5232,49900.0,9397.0,21858.0,3270.0,22.4,35.1,25.4,23.9,22.6,11.3,16.7,83.4,7.4,0.0,0.5,0.3,8.4,29.3,3260,76.8,9.5,13.5,0.2,3.1


In [41]:
merge.loc[105:105]

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
105,1003011601,Alabama,Baldwin County,7131,3699,3432,691,6246,99,49,7,0,5232,49900.0,9397.0,21858.0,3270.0,1597,2502,1811,1704,1611,805,1190,5947,527,0,35,21,599,2089,232470,5476,677,962,14,221,36580,0.01071


In [42]:
int(25.4/100 * 7131)

1811

In [43]:
int(11.3/100 * 7131)

805

In [44]:
assert merge['Professional'][105] == 1811

In [45]:
assert merge['Construction'][105] == 805

#### Mutiply each row by residential ratio

In [46]:
newcol = ['TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment']

In [47]:
merge[newcol] = merge[newcol].multiply(merge['res_ratio'], axis = 'index')

In [48]:
merge[newcol] = merge[newcol].astype(int)

In [49]:
merge.head(15)

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,res_ratio
0,1001020100,Alabama,Autauga County,1845,899,946,44,1592,95,0,22,0,1407,67826,14560,33018,6294,197,383,710,287,420,199,228,1737,60,0,9,0,38,452,16254,1368,391,83,0,84,36067,1.000000
1,1001020200,Alabama,Autauga County,61,32,28,0,25,33,0,0,0,46,1166,107,536,69,13,21,18,15,14,3,9,55,5,0,0,0,0,13,522,46,9,5,0,2,36008,0.028243
2,1001020200,Alabama,Autauga County,2110,1134,976,22,877,1149,0,20,0,1605,40120,3711,18459,2383,472,755,643,524,482,132,324,1909,191,0,0,9,0,468,17982,1601,315,189,0,70,36067,0.971757
3,1001020300,Alabama,Autauga County,3385,1533,1852,270,2078,897,20,23,13,2480,46806,9496,21236,2562,497,714,944,656,1127,335,324,2988,284,0,33,27,50,781,50165,2481,714,162,23,159,36067,1.000000
4,1001020400,Alabama,Autauga County,108,51,57,10,87,7,0,0,0,83,1427,111,716,81,2,1,31,18,28,9,21,89,12,0,1,3,2,28,2015,82,21,4,0,6,36067,0.025543
5,1001020400,Alabama,Autauga County,4158,1949,2208,398,3338,294,20,7,0,3173,54467,4257,27351,3108,95,70,1205,689,1071,378,810,3421,464,0,62,119,86,1076,76880,3151,818,187,0,253,36066,0.974457
6,1001020500,Alabama,Autauga County,7856,3984,3871,70,6087,1288,0,242,0,5699,53723,11371,29095,8440,957,1405,3833,1084,1609,274,1052,6826,879,0,62,22,54,1649,376081,5609,1892,353,0,180,36066,0.788390
7,1001020500,Alabama,Autauga County,2100,1065,1035,18,1627,344,0,64,0,1523,14364,3040,7779,2256,256,375,1024,289,430,73,281,1825,235,0,16,6,14,441,100559,1499,506,94,0,48,36068,0.210806
8,1001020500,Alabama,Autauga County,8,4,3,0,6,1,0,0,0,5,54,11,29,8,0,1,3,1,1,0,1,6,0,0,0,0,0,1,383,5,1,0,0,0,36067,0.000804
9,1001020600,Alabama,Autauga County,3620,1765,1855,108,2559,908,0,0,0,2687,44549,16126,27038,4427,492,731,912,890,1049,293,474,3004,173,0,25,126,289,763,49376,3040,510,68,0,220,36067,1.000000


In [50]:
min(merge['zip']), max(merge['zip'])

(501, 99929)

In [51]:
merge['zip'] = merge['zip'].astype('str').str.zfill(5)

In [52]:
merge = merge.drop(columns = ['res_ratio'])

## Ratios
-return df back to census form

##### Multiply columns by 100

In [53]:
merge.iloc[:,6:11+1] *= 100 

In [54]:
merge.iloc[:,17:36+1] *= 100 

In [55]:
col = ['Hispanic','White', 'Black', 'Native', 'Asian', 'Pacific','Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment']

##### Divide columns by Total Population

In [56]:
merge[col] = merge[col].divide(merge['TotalPop'], axis = 'index')

In [57]:
merge[col] = merge[col].round(1)

##### Check

In [58]:
merge.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.1,0.0,1.2,0.0,1407,67826,14560,33018,6294,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.1,3.3,0.0,0.5,0.0,2.1,24.5,881.0,74.1,21.2,4.5,0.0,4.6,36067
1,1001020200,Alabama,Autauga County,61,32,28,0.0,41.0,54.1,0.0,0.0,0.0,46,1166,107,536,69,21.3,34.4,29.5,24.6,23.0,4.9,14.8,90.2,8.2,0.0,0.0,0.0,0.0,21.3,855.7,75.4,14.8,8.2,0.0,3.3,36008
2,1001020200,Alabama,Autauga County,2110,1134,976,1.0,41.6,54.5,0.0,0.9,0.0,1605,40120,3711,18459,2383,22.4,35.8,30.5,24.8,22.8,6.3,15.4,90.5,9.1,0.0,0.0,0.4,0.0,22.2,852.2,75.9,14.9,9.0,0.0,3.3,36067
3,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806,9496,21236,2562,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482.0,73.3,21.1,4.8,0.7,4.7,36067
4,1001020400,Alabama,Autauga County,108,51,57,9.3,80.6,6.5,0.0,0.0,0.0,83,1427,111,716,81,1.9,0.9,28.7,16.7,25.9,8.3,19.4,82.4,11.1,0.0,0.9,2.8,1.9,25.9,1865.7,75.9,19.4,3.7,0.0,5.6,36067


## Map Zip Codes to City

In [59]:
uszip.columns

Index(['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'zcta',
       'parent_zcta', 'population', 'density', 'county_fips', 'county_name',
       'county_weights', 'county_names_all', 'county_fips_all', 'imprecise',
       'military', 'timezone'],
      dtype='object')

In [60]:
uszip = uszip[['zip', 'city', 'state_name', 'state_id', 'county_name']]

In [61]:
merge = merge.merge(uszip, on = 'zip')

In [62]:
merge.shape

(159684, 42)

In [63]:
merge.head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip,city,state_name,state_id,county_name
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.1,0.0,1.2,0.0,1407,67826,14560,33018,6294,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.1,3.3,0.0,0.5,0.0,2.1,24.5,881.0,74.1,21.2,4.5,0.0,4.6,36067,Prattville,Alabama,AL,Autauga
1,1001020200,Alabama,Autauga County,2110,1134,976,1.0,41.6,54.5,0.0,0.9,0.0,1605,40120,3711,18459,2383,22.4,35.8,30.5,24.8,22.8,6.3,15.4,90.5,9.1,0.0,0.0,0.4,0.0,22.2,852.2,75.9,14.9,9.0,0.0,3.3,36067,Prattville,Alabama,AL,Autauga
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806,9496,21236,2562,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482.0,73.3,21.1,4.8,0.7,4.7,36067,Prattville,Alabama,AL,Autauga
3,1001020400,Alabama,Autauga County,108,51,57,9.3,80.6,6.5,0.0,0.0,0.0,83,1427,111,716,81,1.9,0.9,28.7,16.7,25.9,8.3,19.4,82.4,11.1,0.0,0.9,2.8,1.9,25.9,1865.7,75.9,19.4,3.7,0.0,5.6,36067,Prattville,Alabama,AL,Autauga
4,1001020500,Alabama,Autauga County,8,4,3,0.0,75.0,12.5,0.0,0.0,0.0,5,54,11,29,8,0.0,12.5,37.5,12.5,12.5,0.0,12.5,75.0,0.0,0.0,0.0,0.0,0.0,12.5,4787.5,62.5,12.5,0.0,0.0,0.0,36067,Prattville,Alabama,AL,Autauga


In [64]:
merge.columns

Index(['TractId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'zip', 'city',
       'state_name', 'state_id', 'county_name'],
      dtype='object')

In [65]:
columns = ['TractId', 'city', 'state_id', 'State', 'County', 'county_name', 
           'TotalPop', 'Men', 'Women', 'Hispanic','White', 'Black', 'Native', 
           'Asian', 'Pacific', 'VotingAgeCitizen','Income', 'IncomeErr', 
           'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty', 
           'Professional', 'Service', 'Office', 'Construction', 'Production', 
           'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 
           'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork', 
           'SelfEmployed', 'FamilyWork', 'Unemployment', 'zip', ]

In [66]:
merge = merge[columns]

In [67]:
merge.head()

,TractId,city,state_id,State,County,county_name,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,zip
0,1001020100,Prattville,AL,Alabama,Autauga County,Autauga,1845,899,946,2.4,86.3,5.1,0.0,1.2,0.0,1407,67826,14560,33018,6294,10.7,20.8,38.5,15.6,22.8,10.8,12.4,94.1,3.3,0.0,0.5,0.0,2.1,24.5,881.0,74.1,21.2,4.5,0.0,4.6,36067
1,1001020200,Prattville,AL,Alabama,Autauga County,Autauga,2110,1134,976,1.0,41.6,54.5,0.0,0.9,0.0,1605,40120,3711,18459,2383,22.4,35.8,30.5,24.8,22.8,6.3,15.4,90.5,9.1,0.0,0.0,0.4,0.0,22.2,852.2,75.9,14.9,9.0,0.0,3.3,36067
2,1001020300,Prattville,AL,Alabama,Autauga County,Autauga,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806,9496,21236,2562,14.7,21.1,27.9,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482.0,73.3,21.1,4.8,0.7,4.7,36067
3,1001020400,Prattville,AL,Alabama,Autauga County,Autauga,108,51,57,9.3,80.6,6.5,0.0,0.0,0.0,83,1427,111,716,81,1.9,0.9,28.7,16.7,25.9,8.3,19.4,82.4,11.1,0.0,0.9,2.8,1.9,25.9,1865.7,75.9,19.4,3.7,0.0,5.6,36067
4,1001020500,Prattville,AL,Alabama,Autauga County,Autauga,8,4,3,0.0,75.0,12.5,0.0,0.0,0.0,5,54,11,29,8,0.0,12.5,37.5,12.5,12.5,0.0,12.5,75.0,0.0,0.0,0.0,0.0,0.0,12.5,4787.5,62.5,12.5,0.0,0.0,0.0,36067


In [68]:
merge.columns = merge.columns.str.upper()

In [69]:
merge.to_csv('merge.csv')

In [70]:
!cp merge.csv "drive/My Drive/"

##  CSVs - Group by Cities

In [71]:
merge.columns

Index(['TRACTID', 'CITY', 'STATE_ID', 'STATE', 'COUNTY', 'COUNTY_NAME',
       'TOTALPOP', 'MEN', 'WOMEN', 'HISPANIC', 'WHITE', 'BLACK', 'NATIVE',
       'ASIAN', 'PACIFIC', 'VOTINGAGECITIZEN', 'INCOME', 'INCOMEERR',
       'INCOMEPERCAP', 'INCOMEPERCAPERR', 'POVERTY', 'CHILDPOVERTY',
       'PROFESSIONAL', 'SERVICE', 'OFFICE', 'CONSTRUCTION', 'PRODUCTION',
       'DRIVE', 'CARPOOL', 'TRANSIT', 'WALK', 'OTHERTRANSP', 'WORKATHOME',
       'MEANCOMMUTE', 'EMPLOYED', 'PRIVATEWORK', 'PUBLICWORK', 'SELFEMPLOYED',
       'FAMILYWORK', 'UNEMPLOYMENT', 'ZIP'],
      dtype='object')

### Diversity

In [72]:
diversity_df = merge[['HISPANIC', 'WHITE', 'BLACK', 'NATIVE','ASIAN', 'PACIFIC', 'CITY', 'STATE_ID']].groupby(['CITY', 'STATE_ID'],as_index=False).mean()
diversity_df[['HISPANIC', 'WHITE', 'BLACK', 'NATIVE','ASIAN', 'PACIFIC']] = diversity_df[['HISPANIC', 'WHITE', 'BLACK', 'NATIVE','ASIAN', 'PACIFIC']].round(1)
diversity_df.to_csv("diversity_df.csv",index=False)
diversity_df.head()

,CITY,STATE_ID,HISPANIC,WHITE,BLACK,NATIVE,ASIAN,PACIFIC
0,Aaronsburg,PA,1.0,97.4,0.0,0.0,0.7,0.0
1,Abbeville,AL,3.6,64.2,31.4,0.0,0.0,0.0
2,Abbeville,GA,2.7,58.7,37.1,0.4,0.0,0.0
3,Abbeville,LA,3.9,77.0,14.2,0.3,2.1,0.0
4,Abbeville,MS,4.7,60.9,31.3,0.0,0.8,0.0


### Income Per Capita

In [73]:
income_per_capita_df = merge[['INCOME', 'INCOMEERR','INCOMEPERCAP', 'INCOMEPERCAPERR','CITY', 'STATE_ID']].groupby(['CITY', 'STATE_ID'],as_index=False).mean()
income_per_capita_df[['INCOME', 'INCOMEERR','INCOMEPERCAP', 'INCOMEPERCAPERR']] = income_per_capita_df[['INCOME', 'INCOMEERR','INCOMEPERCAP', 'INCOMEPERCAPERR']].round(1)
income_per_capita_df.to_csv("income_per_capita_df.csv",index=False)
income_per_capita_df.head()

,CITY,STATE_ID,INCOME,INCOMEERR,INCOMEPERCAP,INCOMEPERCAPERR
0,Aaronsburg,PA,10258.0,1258.0,4516.0,324.0
1,Abbeville,AL,27895.8,6168.8,15016.5,2161.2
2,Abbeville,GA,14165.7,3522.0,5631.3,1339.0
3,Abbeville,LA,26087.2,5041.7,12330.7,1597.9
4,Abbeville,MS,7885.3,1314.7,4145.0,634.3


### Poverty

In [74]:
poverty_df = merge[['POVERTY', 'CHILDPOVERTY','CITY', 'STATE_ID']].groupby(['CITY', 'STATE_ID'],as_index=False).sum()
poverty_df.to_csv("poverty_df.csv",index=False)
poverty_df.head()

,CITY,STATE_ID,POVERTY,CHILDPOVERTY
0,Aaronsburg,PA,16.4,31.6
1,Abbeville,AL,71.5,126.0
2,Abbeville,GA,75.2,100.5
3,Abbeville,LA,201.7,251.2
4,Abbeville,MS,58.8,87.8


### Industries

In [75]:
industry_df = merge[['PROFESSIONAL', 'SERVICE', 'OFFICE', 'CONSTRUCTION', 'PRODUCTION','CITY', 'STATE_ID']].groupby(['CITY', 'STATE_ID'],as_index=False).sum()
industry_df.to_csv("industry_df.csv",index=False)
industry_df.head()

,CITY,STATE_ID,PROFESSIONAL,SERVICE,OFFICE,CONSTRUCTION,PRODUCTION
0,Aaronsburg,PA,27.5,21.2,18.9,15.5,16.6
1,Abbeville,AL,109.7,72.6,77.0,39.3,99.7
2,Abbeville,GA,86.5,76.2,44.3,42.2,49.3
3,Abbeville,LA,288.6,184.8,277.4,185.0,145.8
4,Abbeville,MS,108.2,64.9,46.0,25.8,51.9


### Transportation

In [76]:
trasportation_df = merge[['DRIVE', 'CARPOOL', 'TRANSIT', 'WALK', 'OTHERTRANSP', 'WORKATHOME','MEANCOMMUTE','CITY', 'STATE_ID']].groupby(['CITY', 'STATE_ID'],as_index=False).mean()
trasportation_df[['DRIVE', 'CARPOOL', 'TRANSIT', 'WALK', 'OTHERTRANSP', 'WORKATHOME','MEANCOMMUTE']] = trasportation_df[['DRIVE', 'CARPOOL', 'TRANSIT', 'WALK', 'OTHERTRANSP', 'WORKATHOME','MEANCOMMUTE']].round(1)
trasportation_df.to_csv("trasportation_df.csv",index=False)
trasportation_df.head()

,CITY,STATE_ID,DRIVE,CARPOOL,TRANSIT,WALK,OTHERTRANSP,WORKATHOME,MEANCOMMUTE
0,Aaronsburg,PA,69.7,12.4,0.3,3.8,3.6,10.1,27.7
1,Abbeville,AL,85.1,9.2,0.0,1.0,1.7,2.6,26.9
2,Abbeville,GA,86.5,7.0,0.0,1.7,1.0,3.2,27.3
3,Abbeville,LA,80.0,10.7,0.1,3.3,1.6,2.8,29.6
4,Abbeville,MS,83.7,11.7,0.4,0.8,0.4,1.5,25.7


### Employment

In [77]:
employment_df = merge[['EMPLOYED', 'UNEMPLOYMENT','CITY', 'STATE_ID']].groupby(['CITY', 'STATE_ID'],as_index=False).sum()
employment_df.to_csv('employment_df.csv', index=False)
employment_df.head()

,CITY,STATE_ID,EMPLOYED,UNEMPLOYMENT
0,Aaronsburg,PA,2669.4,3.1
1,Abbeville,AL,3341.3,31.0
2,Abbeville,GA,1754.9,18.0
3,Abbeville,LA,22976.5,76.5
4,Abbeville,MS,7629.2,17.3


### Employment Type

In [78]:
employment_ratio_df = merge[['PRIVATEWORK', 'PUBLICWORK', 'SELFEMPLOYED','FAMILYWORK','CITY', 'STATE_ID']].groupby(['CITY', 'STATE_ID'],as_index=False).mean()
employment_ratio_df[['PRIVATEWORK', 'PUBLICWORK', 'SELFEMPLOYED','FAMILYWORK']] = employment_ratio_df[['PRIVATEWORK', 'PUBLICWORK', 'SELFEMPLOYED','FAMILYWORK']].round(1)
employment_ratio_df.to_csv('employment_ratio_df.csv', index=False)
employment_ratio_df.head()

,CITY,STATE_ID,PRIVATEWORK,PUBLICWORK,SELFEMPLOYED,FAMILYWORK
0,Aaronsburg,PA,76.6,10.9,12.4,0.0
1,Abbeville,AL,79.6,14.8,5.4,0.0
2,Abbeville,GA,59.3,30.3,9.8,0.2
3,Abbeville,LA,80.9,12.3,5.4,0.0
4,Abbeville,MS,71.8,21.0,6.6,0.0
